In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### we will be building a tree to decide whether the animal is dog or cat

Therefore, we have two sets:

- `X_train`: for each example, contains 3 features:
            - Ear Shape (1 if pointy, 0 otherwise)
            - Face Shape (1 if round, 0 otherwise)
            - Whiskers (1 if present, 0 otherwise)
            
- `y_train`: whether the animal is a cat
            - 1 if the animal is a cat
            - 0 otherwise

In [2]:
X_train = np.array([[1, 1, 1],
[0, 0, 1],
 [0, 1, 0],
 [1, 0, 1],
 [1, 1, 1],
 [1, 1, 0],
 [0, 0, 0],
 [1, 1, 0],
 [0, 1, 0],
 [0, 1, 0]])

y_train = np.array([1, 1, 0, 0, 1, 1, 0, 1, 0, 0])

### Information Gain
in a decision tree, we decide if a node will be split or not by looking at the **information gain** that split would give us. (Image of video IG)

Where 

$$\text{Information Gain} = H(p_1^\text{node})- \left(w^{\text{left}}H\left(p_1^\text{left}\right) + w^{\text{right}}H\left(p_1^\text{right}\right)\right),$$

and $H$ is the entropy, defined as

$$H(p_1) = -p_1 \text{log}_2(p_1) - (1- p_1) \text{log}_2(1- p_1)$$

In [14]:
def entropy(p1):
    if(p1 == 0 or p1 == 1):
        return 0
    return -p1 * np.log2(p1) - (1-p1) * np.log2(1 - p1)

print(entropy(0.5));

1.0


now let's define the splitting function that will split according to index

In [42]:
def split(X_set,y, featureIndex):
    left_group = []
    right_group = []
    y_left_group = []
    y_right_group = []
    for i,record in enumerate(X_set):
        if record[featureIndex] == 1:
            left_group.append(record)
            y_left_group.append(y[i])
        else:
            right_group.append(record)
            y_right_group.append(y[i])
    return np.array(left_group), np.array(right_group),np.array(y_left_group),np.array(y_right_group)


left,right,y_left,y_right = split(X_train,y_train, 0)
#split(X_train,y_train, 0)
print(f"left is {left} \nright is {right}")

(array([[1, 1, 1],
        [1, 0, 1],
        [1, 1, 1],
        [1, 1, 0],
        [1, 1, 0]]),
 array([[0, 0, 1],
        [0, 1, 0],
        [0, 0, 0],
        [0, 1, 0],
        [0, 1, 0]]),
 array([1, 0, 1, 1, 1]),
 array([1, 0, 0, 0, 0]))

### however
this will help somehow

In [26]:
def split_indices(X_set, featureIndex):
    left_group = []
    right_group = []
    for i,record in enumerate(X_set):
        if record[featureIndex] == 1:
            left_group.append(i)
        else:
            right_group.append(i)
    return left_group, right_group


left,right = split_indices(X_train, 0)
print(f"left is {left} \nright is {right}")

left is [0, 3, 4, 5, 7] 
right is [1, 2, 6, 8, 9]


Now we need another function to compute the weighted entropy in the splitted nodes. As you've seen in the video lecture, we must find:

- $w^{\text{left}}$ and $w^{\text{right}}$, the proportion of animals in **each node**.
- $p^{\text{left}}$ and $p^{\text{right}}$, the proportion of cats in **each split**.

Note the difference between these two definitions!! To illustrate, if we split the root node on the feature of index 0 (Ear Shape), then in the left node, the one that has the animals 0, 3, 4, 5 and 7, we have:

$$w^{\text{left}}= \frac{5}{10} = 0.5 \text{ and } p^{\text{left}} = \frac{4}{5}$$
$$w^{\text{right}}= \frac{5}{10} = 0.5 \text{ and } p^{\text{right}} = \frac{1}{5}$$

In [45]:
def weighted_entropy(X_set,y,left_indices,right_indices):
    
    n_left = len(left_indices)
    n_right = len(right_indices)
    n = len(X_set)
    
    w_left = n_left/n
    w_right = n_right/n
    
    try:
        entropy_left = entropy(sum(y[left_indices])/n_left)
    except:
        entropy_left = 0
    try:
        entropy_right = entropy(sum(y[right_indices])/n_right)
    except:
        entropy_right = 0
    
    return w_left * entropy_left + w_right * entropy_right

print(f"weighted entropy when dividing according to ear shape {weighted_entropy(X_train,y_train,left,right)}")

weighted entropy when dividing according to ear shape 0.0


So, the weighted entropy in the 2 split nodes is 0.72. To compute the **Information Gain** we must subtract it from the entropy in the node we chose to split (in this case, the root node). 

In [28]:
def information_gain(X_set,y,left_indices,right_indices):
    p1_root = sum(y)/len(y)
    return entropy(p1_root) - weighted_entropy(X_set,y,left_indices,right_indices)

print(f"information gain is {information_gain(X_train,y_train,left,right)}")

information gain is 0.2780719051126377


Now, let's compute the information gain if we split the root node for each feature:

In [33]:
for i in range(X_train.shape[1]):
    left,right = split_indices(X_train, i)
    info_gain_i = information_gain(X_train,y_train,left,right)
    print(f"the feature {i} has information gain of {info_gain_i}")

the feature 0 has information gain of 0.2780719051126377
the feature 1 has information gain of 0.034851554559677034
the feature 2 has information gain of 0.12451124978365313


we will turn this into fucntion

In [47]:
def best_split(X_set,y):
    
    best_feature = -1
    mx = -1000000
    for i in range(X_set.shape[1]):
        left,right = split_indices(X_set, i)
        info_gain_i = information_gain(X_set,y,left,right)
        if info_gain_i > mx:
            mx = info_gain_i
            best_feature = i
    return best_feature

print(f"the best feature to split on is {best_split(X_train,y_train)}")

the best feature to split on is 0


### Tree generation code
the overall code needs improvement because it's not so efficient and there is a lot of waste

In [54]:
mx_depth = 2
def tree_gen(X_train,y_train,tree,depth = 0):
    if(depth == mx_depth):
        # adding the leaf node
        return 
    best_feature = best_split(X_train,y_train)
    left,right = split_indices(X_train, best_feature)
    tree.append([depth-1,best_feature,left,right])
    x_left,x_right,y_left,y_right = split(X_train,y_train,best_feature)
    tree_gen(x_left,y_left,tree,depth+1)
    tree_gen(x_right,y_right,tree,depth+1)
    
    return tree

print("first param is the parent, current index is parent + 1 and second param is feature split on, the left and right groups")
tree_gen(X_train,y_train,[])


first param is the parent, current index is parent + 1 and second param is feature split on, the left and right groups


[[-1, 0, [0, 3, 4, 5, 7], [1, 2, 6, 8, 9]],
 [0, 1, [0, 2, 3, 4], [1]],
 [0, 2, [0], [1, 2, 3, 4]]]